### Read float matlab files and convert them to xarray

In [1]:
import os,glob
import numpy as np
import pandas as pd
import xarray as xr
import scipy.io as sio
import gsw
from datetime import datetime,timedelta

In [66]:
def load_matfile(file):
    '''Read Matlab structure files and convert to numpy arrays'''
    return sio.loadmat(file,struct_as_record=True,squeeze_me=True)

def datenum2datetime(datenum):
    '''Convert Matlab datenum to Python Datetime'''
    return datetime.fromordinal(int(datenum)) + timedelta(days=datenum%1) - timedelta(days = 366)

# vecturize datenum2datetime
dn2dt_vec = np.vectorize(lambda x: datenum2datetime(x))

def save_as_xr(floatfile,ab):
    '''Read float files, compose xarray dataset, convert variables, and save as netcdf
    
    last updated: april 11, 2019
    '''
    a = load_file(floatfile)
    
    # compose dataset object
    ds = xr.Dataset({ # define wanted variables here!
        'sigma': (['z','time'],a['A']['Sigma'].flatten()[0]-1000), 
        'n2': (['z','time'],a['A']['N2'].flatten()[0])},
        
        coords = {'pressure':(['z'],a['A']['Pr'].flatten()[0].astype(float)),
                  'z': a['A']['Pr'].flatten()[0].astype(float),
                  'lat':(['time'],a['A']['lat'].flatten()[0]),
                  'lon':(['time'],a['A']['lon'].flatten()[0]),
                  'time':a['A']['Jday_gmt'].flatten()[0].astype(float)},
        
        attrs= {'floatid':a['A']['float'].flatten()[0]}
    )
    # remove nans
    ds = ds.dropna(dim='time',how='all')
    # convert to datetime
    ds.assign_coords(time=(dn2dt_vec(ds.time)))
    # comvert pressure to depth
    ds.assign_coords(z=(gsw.z_from_p(ds.z,ds.lat.mean()).astype(float)))
    # save as netcdf
    ds.to_netcdf(project_path+f"/data/xarray/xr_float_grid_{ds.attrs['floatid']}{ab}.nc" )

There are 6 floats in 2016 and 7 floats in 2017.

#### Process files and write netcdf

In [ ]:
project_path = '/Users/sebastian/Dropbox (MIT)/postdoc/'

In [105]:
for sub in ['a','b']: # loop over two experiments
    floatfiles = glob.glob(project_path+f'data/NIWmatdata/*[{sub}]_grid.mat')
    for file in floatfiles:
        print(file)
        save_as_xr(file,sub)

/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7786a_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7781a_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7787a_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7783a_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7779a_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7788a_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7784b_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7782b_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7785b_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7786b_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7700b_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7780b_grid.mat
/Users/sebastian/Dropbox (MIT)/postdoc/data/NIWmatdata/7701b_grid.mat
